In [ ]:
%pip --quiet install datasets
%pip --quiet install sacrebleu

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.3/106.3 kB 3.1 MB/s eta 0:00:00


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import TensorDataset, DataLoader
import numpy as np
from tqdm import tqdm
from datasets import load_dataset
from nltk.tokenize import word_tokenize
from torch.nn.parameter import Parameter
import sacrebleu

In [ ]:
# Загрузка датасета
raw_datasets = load_dataset("abobster/pushkin_new")

# Запись текста в файл
with open('input.txt', 'w', encoding='utf-8') as f:
    f.write('\n'.join(raw_datasets['train']['text']))

Generating train split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

In [ ]:
# hyperparameters
batch_size = 64
block_size = 256
max_iters = 5000
eval_interval = 50
learning_rate = 3e-4
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print('device', device)
eval_iters = 200
n_embd = 384
dropout = 0.2
# ------------

torch.manual_seed(1337)

with open('input.txt', 'r', encoding='utf-8') as f:
    text = f.read()

chars = sorted(list(set(text)))
vocab_size = len(chars)
stoi = {ch: i for i, ch in enumerate(chars)}
itos = {i: ch for i, ch in enumerate(chars)}
encode = lambda s: [stoi[c] for c in s]
decode = lambda l: ''.join([itos[i] for i in l])

data = torch.tensor(encode(text), dtype=torch.long)
n = int(0.9 * len(data))
train_data = data[:n]
val_data = data[n:]

device cuda


In [ ]:
class Attention(nn.Module):
    def __init__(self, hidden_size):
        super(Attention, self).__init__()
        self.hidden_size = hidden_size
        self.query = nn.Linear(hidden_size, hidden_size)
        self.key = nn.Linear(hidden_size, hidden_size)
        self.value = nn.Linear(hidden_size, hidden_size)
        self.scale = Parameter(torch.FloatTensor([1.0 / (hidden_size ** 0.5)]))

    def forward(self, query, key, value):
        scores = torch.matmul(query, key.transpose(-2, -1))
        scores = scores * self.scale
        attention_weights = F.softmax(scores, dim=-1)
        context = torch.matmul(attention_weights, value)
        return context, attention_weights

class RNNAttentionLanguageModel(nn.Module):
    def __init__(self):
        super(RNNAttentionLanguageModel, self).__init__()
        self.embedding = nn.Embedding(vocab_size, n_embd)
        self.rnn = nn.LSTM(n_embd, n_embd, num_layers=1, dropout=dropout, batch_first=True)
        self.attention = Attention(n_embd)
        self.dropout = nn.Dropout(dropout)
        self.fc = nn.Linear(n_embd, vocab_size)

    def forward(self, x, hidden=None):
        embedded = self.embedding(x)
        output, hidden = self.rnn(embedded, hidden)
        context, _ = self.attention(output, output, output)
        output = self.dropout(context)
        logits = self.fc(output)
        return logits, hidden

    def generate(self, start_tokens, max_new_tokens, temperature=1.0):
        model.eval()
        generated_tokens = start_tokens.clone()

        for _ in range(max_new_tokens):
            logits, _ = model(generated_tokens)
            last_logits = logits[:, -1, :] / temperature
            probabilities = F.softmax(last_logits, dim=-1)
            sampled_token = torch.multinomial(probabilities, 1)
            generated_tokens = torch.cat((generated_tokens, sampled_token), dim=1)

        return generated_tokens

In [ ]:
model = RNNAttentionLanguageModel()
m = model.to(device)
print(sum(p.numel() for p in m.parameters()) / 1e6, 'M parameters')

/usr/local/lib/python3.10/dist-packages/torch/nn/modules/rnn.py:82: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.2 and num_layers=1
  warnings.warn("dropout option adds dropout after all but last "


1.72698 M parameters


In [ ]:
from time import time

def get_batch(split):
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i + block_size] for i in ix])
    y = torch.stack([data[i + 1:i + block_size + 1] for i in ix])
    x, y = x.to(device), y.to(device)
    return x, y

@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, _ = model(X)
            loss = F.cross_entropy(logits.view(-1, vocab_size), Y.view(-1))
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

In [ ]:
def calculate_chrf(reference_texts, hypothesis):
    references = [reference for reference in reference_texts]
    chrf = sacrebleu.sentence_chrf(hypothesis, references).score
    return chrf

In [ ]:
reference_texts = [
    """
    На голубом берегу,
    Где волны шумят,
    Стоял замок высокий,
    Где счастье царит.
    """,
    """
    В поле зрения звезд,
    Ночью сверкающих,
    Мечты рождаются,
    В сердце молодых.
    """,
    """
    Летит стрела в небеса,
    По траектории света,
    Вдоль лазурного свода,
    Встречая рассветы.
    """,
    """
    Среди трав и цветов,
    По дорожке весны,
    Любовь расцветает,
    Как песня весенняя.
    """
]


In [ ]:
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

t0 = time()
for iter in range(max_iters):
    if iter % eval_interval == 0 or iter == max_iters - 1:
        losses = estimate_loss()
        print(f"step {iter}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}, elapsed: {time() - t0:.1f}s")

    xb, yb = get_batch('train')
    logits, _ = model(xb)
    loss = F.cross_entropy(logits.view(-1, vocab_size), yb.view(-1))

    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

step 0: train loss 2.3543, val loss 2.3840, elapsed: 8.1s
step 50: train loss 2.3028, val loss 2.3379, elapsed: 19.1s
step 100: train loss 2.2552, val loss 2.2950, elapsed: 30.0s
step 150: train loss 2.2123, val loss 2.2553, elapsed: 40.9s
step 200: train loss 2.1720, val loss 2.2196, elapsed: 51.7s
step 250: train loss 2.1346, val loss 2.1872, elapsed: 62.5s
step 300: train loss 2.1010, val loss 2.1572, elapsed: 73.2s
step 350: train loss 2.0713, val loss 2.1304, elapsed: 84.0s
step 400: train loss 2.0431, val loss 2.1090, elapsed: 94.8s
step 450: train loss 2.0155, val loss 2.0873, elapsed: 105.7s
step 500: train loss 1.9936, val loss 2.0691, elapsed: 116.5s
step 550: train loss 1.9698, val loss 2.0511, elapsed: 127.4s
step 600: train loss 1.9462, val loss 2.0369, elapsed: 138.2s
step 650: train loss 1.9256, val loss 2.0212, elapsed: 149.0s
step 700: train loss 1.9095, val loss 2.0077, elapsed: 159.8s
step 750: train loss 1.8886, val loss 1.9933, elapsed: 170.7s
step 800: train loss 

In [ ]:
# Вычисление метрики chrF++ после завершения обучения
model.eval()  # Перевести модель в режим оценки

for reference_text in reference_texts:
    start_tokens = torch.zeros((1, 1), dtype=torch.long, device=device)
    generated_tokens = model.generate(start_tokens, 100, temperature=0.8)
    generated_text = decode(generated_tokens.tolist()[0])
    chrf_score = calculate_chrf(reference_text, generated_text)
    print(f"Reference Text: {reference_text}")
    print(f"Generated Text: {generated_text}")
    print(f"chrF++ score: {chrf_score}")

Reference Text: 
    На голубом берегу,
    Где волны шумят,
    Стоял замок высокий,
    Где счастье царит.
    
Generated Text: 
И твой залынет вас привет навел.
Он ужасно, так же спокойный вод!
О сердце колеча разостей;
Моильней
chrF++ score: 5.681818181818181
Reference Text: 
    В поле зрения звезд,
    Ночью сверкающих,
    Мечты рождаются,
    В сердце молодых.
    
Generated Text: 
Стомешь быть мало жить? Но восторгом —
Весне в стороном с своей речей
Не просто он за зоветлася,
Кот
chrF++ score: 5.813953488372093
Reference Text: 
    Летит стрела в небеса,
    По траектории света,
    Вдоль лазурного свода,
    Встречая рассветы.
    
Generated Text: 
Поль были склонница содна,
И в лира советанний брат,
На своего руках неотравы.
Как вы, безмолвно ула
chrF++ score: 5.681818181818181
Reference Text: 
    Среди трав и цветов,
    По дорожке весны,
    Любовь расцветает,
    Как песня весенняя.
    
Generated Text: 
Не во, буйный девы безумной.
Волненье лист и вырны, лесами,
Давно т

In [ ]:
# Предполагаем, что у вас есть экземпляр модели 'model'
# и стартовая последовательность 'start_tokens' (например, начальные символы из вашего текста)
context = torch.zeros((1, 1), dtype=torch.long, device=device)

# Генерация дополнительных токенов
max_new_tokens = 500  # Задайте желаемое количество генерируемых токенов
generated_tokens = model.generate(context, max_new_tokens, temperature=0.6)

# Декодирование сгенерированной последовательности в текст
generated_text = decode(generated_tokens[0].tolist())
print(generated_text)


Спокойный глас постеле слушал.
Но видел он в том улыбко зависти.
Но, может под сенью разлуке.
И ты, долго в полного сладость.
И любовь в высоком снегом
С безомленною принесли
И видеть лет и гордый край,
Не зная к нам в тем ее дневае.
Вот в сем не понимает живо,
Где только в песне странных вор.
Все все равны ответа на смешном
Все думал он ужасно свои вздорил
И с тоску милых нет свет уже речей
И проклятей славы в суде бросами,
Он успел он привек мой дикие страданья,
С нам и камник долго тихонько в


In [ ]:
torch.save(model.state_dict(), 'model_weights.pth')